# About

custome transformer using spacy

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

## sample

In [4]:
t1 = questions[questions["accepted_answer_id"] > 0]
questions_sample = t1.sample(10000, random_state = 20191102)

## corpus

In [7]:
# constructs corpus
# with question id, title, accepted answer id, answer body
def construct_corpus(questions, answers = answers):
    t1 = questions[["id", "title", "tags", "accepted_answer_id"]].rename(columns = {"id" : "q_id", "title" : "q_title"})
    t2 = answers[["id", "body"]].rename(columns = {"id" : "a_id", "body" : "a_body"})
    t3 = t1.merge(t2, left_on = "accepted_answer_id", right_on = "a_id", how = "inner").drop(columns = "a_id")
    return(t3)

In [8]:
corpus = construct_corpus(questions_sample)
corpus.head(3)

,q_id,q_title,tags,accepted_answer_id,a_body
0,17108288,Why is my CSS :hover @keyframes animation not working?,htmlcss3cross-browsercss-animations,17108610.0,"<p>here it is:</p>\r\n\r\n<p>u have to add vendor prefixes</p>\r\n\r\n<pre><code>.twitter-bird {\r\n height: 38px;\r\n width: 37.5px;\r\n background-image: url(""http://dl.dropboxusercontent.com/u/105046436/tw.png"");\r\n}\r\n\r\n.twitter-bird:hover { \r\n -webkit-animation: fly .4s steps(3) infinite;\r\n -moz-animation: fly .4s steps(3) infinite;\r\n -ms-animation: fly .4s steps(3) infinite;\r\n -o-animation: fly .4s steps(3) infinite;\r\n animation: fly .4s steps(3) infinite;\r\n}\r\n\r\n@-webkit-keyframes fly {\r\n from { background-position: 0px; }\r\n to { background-position: -112.5px; }\r\n}\r\n\r\n@-moz-keyframes fly {\r\n from { background-position: 0px; }\r\n to { background-position: -112.5px; }\r\n}\r\n\r\n@-ms-keyframes fly {\r\n from { background-position: 0px; }\r\n to { background-position: -112.5px; }\r\n}\r\n\r\n@-o-keyframes fly {\r\n from { background-position: 0px; }\r\n to { background-position: -112.5px; }\r\n}\r\n\r\n@keyframes fly {\r\n from { background-position: 0px; }\r\n to { background-position: -112.5px; }\r\n}\r\n</code></pre>\r\n\r\n<p>fiddle: <a href=""http://jsfiddle.net/6qHMG/4/"" rel=""noreferrer"">http://jsfiddle.net/6qHMG/4/</a></p>"
1,13656097,Mixing line and scatterplot in ggplot,rggplot2,13656277.0,"<p>As @MattBagg has pointed out, this issue is fixed by moving the <code>aes()</code> for colour out of the main <code>ggplot()</code> call and into the <code>geom_point()</code> call.</p>\r\n\r\n<pre><code>library(ggplot2)\r\n\r\nallData = read.table(header=TRUE,\r\ncolClasses=c(""Date"", ""numeric"", ""character"", ""numeric"", ""numeric""),\r\ntext=""AnnounceDate MarketProbability DealStatus binary BrierScore\r\n2000-04-10 0.3333333 Complete 1 0.2340565\r\n2000-06-14 0.2142857 Complete 1 0.3618200\r\n2000-06-26 0.6846154 Complete 1 0.3690167\r\n2000-06-16 0.1875000 Complete 1 0.4364041\r\n2000-10-05 0.9555556 Complete 1 0.3078432\r\n2000-10-19 0.8500000 Complete 1 0.2670799"")\r\n\r\np1 = ggplot(data=allData, aes(x=AnnounceDate)) +\r\n geom_point(aes(y=MarketProbability, colour=DealStatus)) + \r\n scale_colour_hue(h = c(180,0)) + \r\n geom_line(aes(y=BrierScore))\r\n\r\nggsave(filename=""plot_1.png"", plot=p1, height=2.5, width=5)\r\n</code></pre>\r\n\r\n<p><img src=""https://i.stack.imgur.com/MF9gp.png"" alt=""enter image description here""></p>"
2,395599,Java graphic library for multicoloured text,javagraphics,395679.0,"<p>I'm assuming you're rendering text to an arbitrary component (via paintComponent()) rather than trying to modify the color of text in a JTextPane, JLabel, or other pre-existing widget.</p>\r\n\r\n<p>If this is the case, you should look into using an <a href=""http://java.sun.com/javase/6/docs/api/java/text/AttributedString.html"" rel=""nofollow noreferrer"">AttributedString</a> along with <a href=""http://java.sun.com/javase/6/docs/api/java/awt/font/TextAttribute.html"" rel=""nofollow noreferrer"">TextAttribute</a>. These allow you to assign different styles (color, font, etc.) to various ranges of characters within a string and then render the whole string using Graphics.drawString(...). This way, the underlying graphics subsystem will handle any necessary changes to the graphics state during rendering making your code much more readable, and probably faster.</p>\r\n\r\n<p><a href=""http://academicjava.com/Java_help/Tutorial_Examples_Graphics_12.html"" rel=""nofollow noreferrer"">Here</a> is an example usage.</p>\r\n\r\n<p>Of course, as other have mentioned, you should also be caching your Color objects rather than recreating them over and over.</p>"


# Create and Train Model

## cleaner

In [9]:
from sklearn.base import TransformerMixin
from bs4 import BeautifulSoup

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # gets just the txt, without the html tags
    t1 = BeautifulSoup(text, 'html.parser').get_text()
    # Removing spaces and converting text into lowercase
    t2 = t1.strip().lower()
    return t2

In [19]:
# tests on the cleaner
t1 = predictors()
t2 = corpus.loc[1,"q_title"]

print("===========")
print(t2)
print("===========")
print(clean_text(t2))
print("===========")
print(t1.transform([t2]))

Mixing line and scatterplot in ggplot
mixing line and scatterplot in ggplot
['mixing line and scatterplot in ggplot']


## tokenizer

In [20]:
import string
import spacy

# Create our list of punctuation marks
# TODO more punctuations???
punctuations = string.punctuation

# Create our list of stopwords
# TODO en_core_web_sm ???
nlp = spacy.load("en")
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = spacy.lang.en.English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

## vectorizer

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TODO tweak min doc frequency???
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0)

## train model

In [22]:
from sklearn.pipeline import Pipeline

# Create pipeline using a custom cleaner, and our vectorizer
pipe = Pipeline([
    ("cleaner", predictors()),
    ('vectorizer', vectorizer)
])

# model generation
m1 = pipe.fit_transform(corpus.q_title)

In [23]:
# inspect tokens
tokens = vectorizer.get_feature_names()
print(len(tokens))
print(np.asarray(tokens[1:100]))

111469
['00 and' '00 and ends' '00 in' '00 in every' '00653' '00653 error'
 '00653 error on' '07' '07 25' '07 25 has' '0700' '0700 registration'
 '0700 registration of' '0c' '0px' '0px left' '0px left to' '0x0'
 '0x8876086a' '0x8876086a when' '0x8876086a when running' '10' '10 100'
 '10 100 1000' '10 11' '10 cfchart' '10 cfchart how' '10 in' '10 in log'
 '10 js' '10 matrix' '10 matrix to' '10 of' '10 of the' '10 or'
 '10 or more' '10 python' '10 value' '10 value in' '10 waterline'
 '10 waterline what' '10 years' '10 years of' '100' '100 00' '100 00 in'
 '100 1000' '100 99' '100 99 99' '100 both' '100 both width' '100 entries'
 '100 entries through' '100 in' '100 in mpandroidchart' '100 opaque'
 '100 radius' '100 radius miles' '100 random' '100 random surveys'
 '100 range' '1000' '1000 10' '1000 10 matrix' '1000 90' '1000 and'
 '1000 and 1000' '10000' '10000 milliseconds' '1000k' '1000k instead'
 '1000k instead of' '101' '10140' '10140 nor' '10140 nor curent' '1063'
 '1063 argument' '10

In [24]:
from sklearn.metrics.pairwise import linear_kernel

# helper function to compare a input question to every question in the corpus
def find_similar(vectorizer, tfidf_matrix, question_input, top_n = 3):
    t1 = vectorizer.transform([question_input])
    cosine_similarities = linear_kernel(t1, tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [45]:
# helper function to decorate the results found by the find_similar function
def decorate_answers(similar_qs, corpus = corpus):
    t1 = pd.DataFrame(similar_qs, columns = ["q_index", "cos_similarity"]).set_index(["q_index"])
    t2 = corpus.iloc[t1.index]
    t3 = pd.concat([t1, t2], axis = 1)
    t4 = t3[[
        "cos_similarity"
        , "q_id"
        , "q_title"
        , "tags"
        , "accepted_answer_id"
        , "a_body"
    ]]
    return(t4)

# match questions

## existing question

In [46]:
# match on a question that is an exact match of an existing question
# should contain at least 1 match with cosine similarity == 1
similar_qs = find_similar(
    vectorizer = vectorizer
    , tfidf_matrix = m1
    , question_input = "What is the difference between \r and \n?"
)
similar_qs

[(9406, 0.5465365979311178),
 (2683, 0.5340944929842533),
 (99, 0.5201285716189932)]

In [50]:
t1 = decorate_answers(similar_qs)
t1

,cos_similarity,q_id,q_title,tags,accepted_answer_id,a_body
8822,0.304882,19598522,How do create perpetual animation without freezing?,javascriptanimationwhile-loopconditionalinfinite-loop,19598634.0,"<p>Since Javascript is asynchornous there is no <code>sleep</code> methode. But instead, a good idea is to use <a href=""http://www.w3schools.com/jsref/met_win_setinterval.asp"" rel=""nofollow""><code>setInterval</code></a> or <a href=""http://www.w3schools.com/jsref/met_win_settimeout.asp"" rel=""nofollow""><code>setTimeout</code></a>.</p>\r\n\r\n<p>Without these function, your loop is going to take as most ressources as it can and it can result in a freeze (depending on your browser).</p>"
2353,0.267349,39579175,How do I create two different backgrounds on the same page?,htmlcssbackgroundbackground-color,39579501.0,"<p>the curved separation comes from the fact that the header image has purple in the bottom to create the curve. the actual file can be seen here: <a href=""http://www.zucoffee.com/wp-content/themes/zu-coffee/images/header-bg.gif"" rel=""nofollow"">http://www.zucoffee.com/wp-content/themes/zu-coffee/images/header-bg.gif</a></p>\r\n\r\n<p>This (to my knowledge) is fairly common practice to have one background that serves the entire page, and have another separate background on top of that which serves just the header and/or footer. depending on the style of the site, this may include interesting separations, and I have also seen .pngs used to get the header/footer design to 'fade' into the main background. </p>\r\n\r\n<p>There are obviously many different ways that you can do this, and I'm sure someone will want to add on to my answer, but the way your example site chose was to use the <code>background</code> property on their body and header tags to add their images. a snippet from their css file below:</p>\r\n\r\n<pre><code>body { \r\n padding:0; \r\n margin:0;\r\n background:#4b1259 url(images/tile-bg.gif) repeat-x;\r\n width:100%;\r\n display:table;\r\n font:16px/22px Georgia;\r\n color:#ffffff;\r\n text-align:left; \r\n}\r\n.header-bg { //this is where i found them adding the header image.\r\n margin-bottom:20px;\r\n background:url(images/header-bg.gif) no-repeat center top;\r\n}\r\n</code></pre>"
7321,0.265658,1222713,how do I create a line of arbitrary thickness using Bresenham?,algorithmgraphics,1222785.0,"<p>I think the best way is to draw a rectangle rather than a line since a line with width is a two dimensional object. Tring to draw a set of parallel lines to avoid overdraw (to reduce write bandwidth) and underdraw (missing pixels) would be quite complex. It's not too hard to calculate the corner points of the rectangle from the start and end point and the width.</p>\r\n\r\n<p>So, following a comment below, the process to do this would be:-</p>\r\n\r\n<ol>\r\n<li>Create a rectangle the same length as the required line and width equal to the desired width, so (0,0) to (width,length)</li>\r\n<li>Rotate and translate the rectangles corner coordinates to the desired position using a 2D transformation</li>\r\n<li>Rasterise the rotated rectangle, either using a hardware accelerated renderer (an OpenGL quad for example*) or use a software rasteriser. It can be rendered using a quad rasteriser or as a pair of triangles (top left and bottom right for example).</li>\r\n</ol>\r\n\r\n<p>Note *: If you're using OpenGL you can also do Step 2 at the same time. Of course, using OpenGL does mean understanding OpenGL (big and complex) and this application may make that a tricky thing to implement at such a late stage in development.</p>"


## user input question

In [48]:
# match on a question that is not an existing question
similar_qs = find_similar(
    vectorizer = vectorizer
    , tfidf_matrix = m1
    , question_input = "How do I create a bar chart?"
)
similar_qs

[(8822, 0.30488205766262383),
 (2353, 0.26734856457665773),
 (7321, 0.26565789795496053)]

In [51]:
t1 = decorate_answers(similar_qs)
t1

,cos_similarity,q_id,q_title,tags,accepted_answer_id,a_body
8822,0.304882,19598522,How do create perpetual animation without freezing?,javascriptanimationwhile-loopconditionalinfinite-loop,19598634.0,"<p>Since Javascript is asynchornous there is no <code>sleep</code> methode. But instead, a good idea is to use <a href=""http://www.w3schools.com/jsref/met_win_setinterval.asp"" rel=""nofollow""><code>setInterval</code></a> or <a href=""http://www.w3schools.com/jsref/met_win_settimeout.asp"" rel=""nofollow""><code>setTimeout</code></a>.</p>\r\n\r\n<p>Without these function, your loop is going to take as most ressources as it can and it can result in a freeze (depending on your browser).</p>"
2353,0.267349,39579175,How do I create two different backgrounds on the same page?,htmlcssbackgroundbackground-color,39579501.0,"<p>the curved separation comes from the fact that the header image has purple in the bottom to create the curve. the actual file can be seen here: <a href=""http://www.zucoffee.com/wp-content/themes/zu-coffee/images/header-bg.gif"" rel=""nofollow"">http://www.zucoffee.com/wp-content/themes/zu-coffee/images/header-bg.gif</a></p>\r\n\r\n<p>This (to my knowledge) is fairly common practice to have one background that serves the entire page, and have another separate background on top of that which serves just the header and/or footer. depending on the style of the site, this may include interesting separations, and I have also seen .pngs used to get the header/footer design to 'fade' into the main background. </p>\r\n\r\n<p>There are obviously many different ways that you can do this, and I'm sure someone will want to add on to my answer, but the way your example site chose was to use the <code>background</code> property on their body and header tags to add their images. a snippet from their css file below:</p>\r\n\r\n<pre><code>body { \r\n padding:0; \r\n margin:0;\r\n background:#4b1259 url(images/tile-bg.gif) repeat-x;\r\n width:100%;\r\n display:table;\r\n font:16px/22px Georgia;\r\n color:#ffffff;\r\n text-align:left; \r\n}\r\n.header-bg { //this is where i found them adding the header image.\r\n margin-bottom:20px;\r\n background:url(images/header-bg.gif) no-repeat center top;\r\n}\r\n</code></pre>"
7321,0.265658,1222713,how do I create a line of arbitrary thickness using Bresenham?,algorithmgraphics,1222785.0,"<p>I think the best way is to draw a rectangle rather than a line since a line with width is a two dimensional object. Tring to draw a set of parallel lines to avoid overdraw (to reduce write bandwidth) and underdraw (missing pixels) would be quite complex. It's not too hard to calculate the corner points of the rectangle from the start and end point and the width.</p>\r\n\r\n<p>So, following a comment below, the process to do this would be:-</p>\r\n\r\n<ol>\r\n<li>Create a rectangle the same length as the required line and width equal to the desired width, so (0,0) to (width,length)</li>\r\n<li>Rotate and translate the rectangles corner coordinates to the desired position using a 2D transformation</li>\r\n<li>Rasterise the rotated rectangle, either using a hardware accelerated renderer (an OpenGL quad for example*) or use a software rasteriser. It can be rendered using a quad rasteriser or as a pair of triangles (top left and bottom right for example).</li>\r\n</ol>\r\n\r\n<p>Note *: If you're using OpenGL you can also do Step 2 at the same time. Of course, using OpenGL does mean understanding OpenGL (big and complex) and this application may make that a tricky thing to implement at such a late stage in development.</p>"


## list of user input questions

In [71]:
question_list = [
    'Is there a way to visualize the distribution of my data?',
    'How do I show data on a map?',
    'How can I illustrate changes in my data over time?',
    'Is there a way to show a “heatmap” of my data?',
    'How can I present a picture of a network?',
    'How can I plot a comparison of two data sets?',
    'How can I create a chart without coding?',
    'When should I use a bar chart versus a pie chart?',
    'What is the easiest way to create a diagram of a network?',
    'I need help creating a visualization/diagram/presentation of my data',
    'When should I use a scatter plot?',
    'Data visualization tips',
    'How do I make a network diagram in d3?',
    'How do I plot 2 datasets?',
    'How can I animate a bar chart in Python?',
    'I know how to create a line chart with matplotlib, but how do I do it in R?',
    'What is the easiest way to create a heat map of the US?',
    'How can I animate a choropleth in Tableau?  In PowerBI?  In D3?'
]

def g1(question_list):
    res = []
    res_df = pd.DataFrame()
    for q in question_list:
        similar_qs = find_similar(
            vectorizer = vectorizer
            , tfidf_matrix = m1
            , question_input = q
            , top_n = 5
        )
        t1 = decorate_answers(similar_qs)
        t1["user_input"] = q
        t2 = t1[[
            "user_input"
            , "cos_similarity"
            , "q_id"
            , "q_title"
            , "tags"
            , "accepted_answer_id"
            , "a_body"
        ]]
        res.append(t2)
    res_df = pd.concat(res)
    res_df.sort_values(["user_input", "cos_similarity"], ascending=[True, False])
    return(res_df)

t1 = g1(question_list)

In [72]:
t1

,user_input,cos_similarity,q_id,q_title,tags,accepted_answer_id,a_body
6025,Is there a way to visualize the distribution of my data?,0.276722,13689631,Is there a way to get a place name based on coordinates?,c#geolocationcoordinateswindows-store-appsbing-maps,13696332.0,"<p>I usually store the latitude/longitude coordinates and then use GMaps to look up the location, then ""on a best effort basis"" - look up the place's name using the address - again via Google Maps.</p>\r\n\r\n<pre><code>static string baseUri = \r\n ""http://maps.googleapis.com/maps/api/geocode/xml?latlng={0},{1}&amp;sensor=false"";\r\nstring location = string.Empty;\r\n\r\npublic static void RetrieveFormatedAddress(string lat, string lng)\r\n{\r\n string requestUri = string.Format(baseUri, lat, lng);\r\n\r\n using (WebClient wc = new WebClient())\r\n {\r\n string result = wc.DownloadString(requestUri);\r\n var xmlElm = XElement.Parse(result);\r\n var status = (from elm in xmlElm.Descendants() where \r\n elm.Name == ""status"" select elm).FirstOrDefault();\r\n if (status.Value.ToLower() == ""ok"")\r\n {\r\n var res = (from elm in xmlElm.Descendants() where \r\n elm.Name == ""formatted_address"" select elm).FirstOrDefault();\r\n requestUri = res.Value;\r\n }\r\n }\r\n}\r\n</code></pre>\r\n\r\n<p>Edit:</p>\r\n\r\n<p>Here's a simple version of the reverse:</p>\r\n\r\n<pre><code>public static Coordinate GetCoordinates(string region)\r\n{\r\n using (var client = new WebClient())\r\n {\r\n\r\n string uri = ""http://maps.google.com/maps/geo?q='"" + region + \r\n ""'&amp;output=csv&amp;key=sadfwet56346tyeryhretu6434tertertreyeryeryE1"";\r\n\r\n string[] geocodeInfo = client.DownloadString(uri).Split(',');\r\n\r\n return new Coordinate(Convert.ToDouble(geocodeInfo[2]), \r\n Convert.ToDouble(geocodeInfo[3]));\r\n }\r\n}\r\n\r\npublic struct Coordinate\r\n{\r\n private double lat;\r\n private double lng;\r\n\r\n public Coordinate(double latitude, double longitude)\r\n {\r\n lat = latitude;\r\n lng = longitude;\r\n\r\n }\r\n\r\n public double Latitude { get { return lat; } set { lat = value; } }\r\n public double Longitude { get { return lng; } set { lng = value; } }\r\n\r\n}\r\n</code></pre>"
3289,Is there a way to visualize the distribution of my data?,0.269952,5479972,Mathematica NDSolve: Is there a way to have variable coefficients?,functionvariableswolfram-mathematicaplot,5480072.0,"<p>You should just use <code>SetDelayed</code> (""<code>:=</code>"") instead of <code>Set</code> in the function definition:</p>\r\n\r\n<pre><code>X[\[CapitalDelta]_, \[CapitalOmega]_, \[CapitalGamma]_] := \r\n NDSolve[{\[Rho]eg'[\r\n t] == (I*\[CapitalDelta] - .5*\[CapitalGamma])*\[Rho]eg[t] - \r\n I*.5*\[CapitalOmega]*\[Rho]ee[t] + \r\n I*.5*\[CapitalOmega]*\[Rho]gg[t], \[Rho]ge'[\r\n t] == (-I*\[CapitalDelta] - .5*\[CapitalGamma])*\[Rho]ge[t] + \r\n I*.5*\[CapitalOmega]\[Conjugate]*\[Rho]ee[t] - \r\n I*.5*\[CapitalOmega]\[Conjugate]*\[Rho]gg[t], \[Rho]ee'[\r\n t] == -I*.5*\[CapitalOmega]\[Conjugate]*\[Rho]eg[t] + \r\n I*.5*\[CapitalOmega]*\[Rho]ge[t] - \[CapitalGamma]*\[Rho]ee[\r\n t], \[Rho]gg'[t] == \r\n I*.5*\[CapitalOmega]\[Conjugate]*\[Rho]eg[t] - \r\n I*.5*\[CapitalOmega]*\[Rho]ge[t] + \[CapitalGamma]*\[Rho]ee[\r\n t], \[Rho]ee[0] == 0, \[Rho]gg[0] == 1, \[Rho]ge[0] == \r\n 0, \[Rho]eg[0] == \r\n 0}, {\[Rho]ee, \[Rho]eg, \[Rho]ge, \[Rho]gg}, {t, 0, 12}];\r\nPlot[Evaluate[{\[Rho]ee[t] /. X[5, 2, 6], \[Rho]ee[t] /. \r\n X[2, 6, 17]}], {t, 0, 10}, PlotRange -&gt; {0, 1}]\r\n</code></pre>"
1316,Is there a way to visualize the distribution of my data?,0.264565,6748937,Is there a way to colorize the git add --interactive mode?,gitcommand-line-interface,6748991.0,"<p>In your <code>.gitconfig</code>:</p>\r\n\r\n<pre><code>[color]\r\n interactive = true\r\n</code></pre>\r\n\r\n<p>I'm not sure <em>exactly</em> how this will get colorized, but that's the builtin option.</p>"
5660,Is there a way to visualize the distribution of my data?,0.260371,48014162,Is there a way to use VSTS Variable 

In [73]:
t1.user_input.value_counts()

Is there a way to show a “heatmap” of my data?                                 5
I need help creating a visualization/diagram/presentation of my data           5
When should I use a scatter plot?                                              5
How can I animate a bar chart in Python?                                       5
How can I plot a comparison of two data sets?                                  5
How can I animate a choropleth in Tableau?  In PowerBI?  In D3?                5
How can I present a picture of a network?                                      5
Is there a way to visualize the distribution of my data?                       5
What is the easiest way to create a heat map of the US?                        5
How do I make a network diagram in d3?                                         5
What is the easiest way to create a diagram of a network?                      5
When should I use a bar chart versus a pie chart?                              5
How do I show data on a map?